In [4]:
import pandas as pd
import numpy as np
from pandas import Series,DataFrame
import matplotlib.pyplot as plt
import seaborn as sns
import time
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")

In [18]:
path_train = "./demo.csv"  # 训练文件
path_test  = "./demo.csv"  # 测试文件

df = pd.read_csv(path_train,dtype={'TERMINALNO':np.uint16,'TIME':np.uint32,'TRIP_ID':np.uint8,'LONGITUDE':np.float32,'LATITUDE':np.float32,'DIRECTION':np.int16,'HEIGHT':np.float32,'SPEED':np.float32,'CALLSTATE':np.uint8,'Y':np.float16})
df['ftime'] = df['TIME'].apply(lambda x: time.strftime('%Y-%m-%d %H:%M:%S',time.localtime(x)))
df['ftime'] = pd.to_datetime(df['ftime'])
df['hour'] = df['ftime'].dt.hour.astype(np.uint8)
df['weekday'] = df['ftime'].dt.weekday.astype(np.uint8)
df['day'] = df['ftime'].dt.day.astype(np.uint8)
df['month'] = df['ftime'].dt.month.astype(np.uint8)

X = df.iloc[:,:-1]
y = df['Y']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=33, test_size=0.20)

df = pd.read_csv(path_test,dtype={'TERMINALNO':np.uint16,'TIME':np.uint32,'TRIP_ID':np.uint8,'LONGITUDE':np.float32,'LATITUDE':np.float32,'DIRECTION':np.int16,'HEIGHT':np.float32,'SPEED':np.float32,'CALLSTATE':np.uint8,'Y':np.float16})
X_valid = df

In [ ]:
def data_transformation(df):
    df_owner = df.groupby(['TERMINALNO'])['Y'].agg({'Y':np.mean,'counts':'count'}).reset_index()
    return df_owner

def sfc1(df): # 构造速度特征:均值及标准差,返回车主ID,标准差,均值
    df_speed = df.groupby(['TERMINALNO'])['SPEED'].agg({'sstd':np.std,'smean':np.mean}).reset_index()
    return df_speed

def hfc1(df): # 构造海拔特征:均值及标准差,返回车主ID,标准差,均值
    df_hight = df.groupby(['TERMINALNO'])['HEIGHT'].agg({'hstd':np.std,'hmean':np.mean}).reset_index()
    return df_hight

def tfc1(df): # 构造时间特征: